In [4]:
import pandas as pd

stat_links = pd.read_csv('S:\Projects/anime\ML\links/stats.csv')
stat_links.head(5)

,Unnamed: 0,anime_stat
0,0,https://myanimelist.net/anime/16498/Shingeki_n...
1,1,https://myanimelist.net/anime/1535/Death_Note/...
2,2,https://myanimelist.net/anime/5114/Fullmetal_A...
3,3,https://myanimelist.net/anime/30276/One_Punch_...
4,4,https://myanimelist.net/anime/11757/Sword_Art_...


In [5]:
stat_links.columns = ['id','anime_stat']

In [6]:
stat_links = stat_links.drop('id',axis=1)

In [7]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait

driver = webdriver.Chrome("S:\Projects/anime\ML\chromedriver\chromedriver_win32\chromedriver.exe")
wait = WebDriverWait(driver, 7)

C:\Users\mvspr\AppData\Local\Temp\ipykernel_58932\4269025399.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("S:\Projects/anime\ML\chromedriver\chromedriver_win32\chromedriver.exe")


In [8]:
def collect_user():
    links = []
    link = driver.find_elements(By.CSS_SELECTOR, 'table[class="table-recently-updated"] td[class="borderClass"] a')
    for item in link:
        links.append(item.get_attribute('href'))

    up_links = []
    for i in range(0,150):
        if (i%2 == 0):
            up_links.append(links[i])

    return up_links

In [9]:
def collect_score():
    data = []
    link = driver.find_elements(By.CSS_SELECTOR, 'table[class="table-recently-updated"] td[class="borderClass ac"]')
    for item in link:
        data.append(item.text)

    re_data = []
    for i in range(0,300):
        if i%4 == 0:
            re_data.append([data[i]])

    return re_data

In [10]:
def sampling(link_score):
    anime_score = []
    sampling = link_score.to_numpy()
    for d in sampling:
        if(d[1].isdigit()):
            anime_score.append(d)

    anime_score = pd.DataFrame(anime_score)
    anime_score.columns = ['links','scores']

    return anime_score

In [12]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

count = 0
for sl in stat_links['anime_stat']:
    driver.get(sl)
    
    links = collect_user()
    scores = collect_score()

    links = pd.DataFrame(links)
    links.columns = ['links']
    scores = pd.DataFrame(scores)
    scores.columns = ['scores']

    link_score = pd.concat([links,scores],axis=1,join='inner')

    sampled_score = sampling(link_score)

    title = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'h1[class="title-name h1_bold_none"]')))
    title = title.text

    i = 0
    for sc in sampled_score['links']:
        driver.get(sc)
        user_stats = []
        score = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR,'div[class="stats anime"]')))
        score = score.text
        score = score.split('\n')
        for s in score:
            s = s.split(' ')
            user_stats.append(s)

        user_data = [i,title,sampled_score['scores'][0],user_stats[1][1], user_stats[1][4],user_stats[3][0],user_stats[5][0],user_stats[7][0],user_stats[9][0],user_stats[11][0],user_stats[13][0],user_stats[15][0],user_stats[17][0]]
        for j in range(2,13):
            user_data[j] = user_data[j].replace(',','')
            user_data[j] = float(user_data[j])

        u_data = []
        for j in range(2):
            u_data.append(user_data)

        u_data = pd.DataFrame(u_data)
        u_data.columns = ['id','Title','Score','Days','Mean Score', 'Watching', 'Completed', 'On Hold', 'Dropped', 'Plan to Watch', 'Total Entries', 'Rewatched', 'Episodes']
        u_data = u_data.drop_duplicates()
        
        print(f'users : {i}')
        i = i+1

    
    
    print(f'animes : {count}')
    count = count + 1


AttributeError: 'int' object has no attribute 'replace'

In [ ]:
i = 0
print(f'users {i}')

users 0
